In [41]:
# have stack.tif file in the same directory as this file for it to work
# additional functionalities to add: scroll bar

%matplotlib qt

import numpy as np
from PIL import Image, ImageSequence
import matplotlib.pyplot as plt
import cv2


# Load image
im = Image.open('stack.tif')

# Find no. of slices
no_frames = im.n_frames

# Create separate tif files for each slice
for i in range(no_frames):
    im.seek(i)
    im.save('page_%s.tif'%(i,))
    
# create list of individual slices
images = []
for i in range(no_frames):
    temp_im = Image.open('page_%s.tif'%(i,))
    imdata = temp_im.getdata()
    x = np.reshape(imdata, (temp_im.height, temp_im.width))
    x = np.asarray(x, dtype=np.uint8)
    images.append(x)

# convert to numpy array
images_array = np.asarray(images)
np.shape(images_array) # 15, 1024, 1024
np.shape(images_array[0][0]) # 1024
type(images_array)


# create arrays of what should be red, blue and white
combined_channels = np.zeros((5, 1024, 1024))
combined_red = np.zeros((5, 1024, 1024))
combined_blue = np.zeros((5, 1024, 1024))
combined_white = np.zeros((5, 1024, 1024))
for i in range(0, 15, 3):
    combined_channels[i//3] = (images_array[i] + images_array[i+1] + images_array[i+2])
    combined_red[i//3] = (images_array[i])
    combined_blue[i//3] = images_array[i+1]
    combined_white[i//3] = images_array[i+2]

#rearrange shape for scrolling in correct plane (z)
new_stacked = np.dstack(combined_channels)
new_red = np.dstack(combined_red)
new_blue = np.dstack(combined_blue)
new_white = np.dstack(combined_white)  

# !!! now need to convert to r g and white

combined_colours = new_red + new_blue + new_white


# Fixing random state for reproducibility
np.random.seed(19680801)

# Allow scrolling through slices


class IndexTracker:
    def __init__(self, ax, X):
        self.ax = ax
        ax.set_title('use scroll wheel to navigate images')

        self.X = X
        rows, cols, self.slices = X.shape
        self.ind = self.slices//2

        self.im = ax.imshow(self.X[:, :, self.ind])
        self.update()

    def on_scroll(self, event):
        print("%s %s" % (event.button, event.step))
        if event.button == 'up':
            self.ind = (self.ind + 1) % self.slices
        else:
            self.ind = (self.ind - 1) % self.slices
        self.update()

    def update(self):
        self.im.set_data(self.X[:, :, self.ind])
        self.ax.set_ylabel('slice %s' % self.ind)
        self.im.axes.figure.canvas.draw()


fig, ax = plt.subplots(1, 1)

# reshape array to be able to scroll through images

tracker = IndexTracker(ax, img_rgb)


fig.canvas.mpl_connect('scroll_event', tracker.on_scroll)
plt.show()

In [43]:
# go from here to investigate colourmap data
# use dir(variable) to find data and attributes associated with that image
# idea - reshape the array into the number of channels

# !!! just need to shift all the pixel data to the left by 1

from PIL import Image
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt
imageObj = Image.open('stack.tif')
cmapdata = imageObj.getpalette()
imdata = imageObj.getdata()
print("The colormap data:", np.shape(cmapdata))
dir(imageObj)
cmapNx3 = np.reshape(cmapdata,(256,3), order='C')
cmapNx3n = np.asarray(cmapNx3, dtype=float)/255.0
cmap = colors.ListedColormap(cmapNx3n, name='stack_colormap')
plt.imshow(imageObj, cmap=cmap, norm=colors.NoNorm())
plt.colorbar()
plt.axis('Off')
cmapNx3n



The colormap data: (768,)


array([[0.        , 0.00392157, 0.00784314],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.00784314, 0.01176471],
       [0.        , 0.01176471, 0.01568627],
       [0.        , 0.01568627, 0.01960784],
       [0.        , 0.01568627, 0.02352941],
       [0.        , 0.01960784, 0.02745098],
       [0.        , 0.02352941, 0.03137255],
       [0.        , 0.02745098, 0.03529412],
       [0.        , 0.03137255, 0.03921569],
       [0.        , 0.03137255, 0.04313725],
       [0.        , 0.03529412, 0.04705882],
       [0.        , 0.03921569, 0.05098039],
       [0.        , 0.04313725, 0.05490196],
       [0.        , 0.04705882, 0.05882353],
       [0.        , 0.04705882, 0.0627451 ],
       [0.        , 0.05098039, 0.06666667],
       [0.        , 0.05490196, 0.07058824],
       [0.        , 0.05882353, 0.0745098 ],
       [0.        , 0.0627451 , 0.07843137],
       [0.        , 0.0627451 , 0.08235294],
       [0.